<a href="https://colab.research.google.com/github/abiralchy0987/movie_recommendation_system/blob/main/Accuracy_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Mean Absolute Error (MAE) and Root Mean Squared Error (RMSE)
from surprise import accuracy

# Assuming you have already trained your SVD model and have predictions
predictions = svd.test(testset)

# Calculate MAE and RMSE
mae = accuracy.mae(predictions)
rmse = accuracy.rmse(predictions)

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

ModuleNotFoundError: No module named 'surprise'

In [9]:
# Precision, Recall, and F1-Score

from surprise import accuracy

# Assuming you have already trained your SVD model and have predictions
predictions = svd.test(testset)

# Calculate MAE and RMSE
mae = accuracy.mae(predictions)
rmse = accuracy.rmse(predictions)

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

ModuleNotFoundError: No module named 'surprise'

In [ ]:
# Hit Rate(cf and cbf)

def hit_rate(recommended_items, relevant_items):
    hits = 0
    for user in relevant_items:
        if any(item in recommended_items[user] for item in relevant_items[user]):
            hits += 1
    return hits / len(relevant_items)

# Example usage
recommended_items = {1: [101, 102, 103], 2: [104, 105, 106]}  # User ID: [Recommended Items]
relevant_items = {1: [101, 107], 2: [104, 108]}  # User ID: [Relevant Items]

hit_rate_value = hit_rate(recommended_items, relevant_items)
print(f'Hit Rate: {hit_rate_value}')

In [ ]:
# Mean Reciprocal Rank (MRR)

def mean_reciprocal_rank(recommended_items, relevant_items):
    reciprocal_ranks = []
    for user in relevant_items:
        for rank, item in enumerate(recommended_items[user], 1):
            if item in relevant_items[user]:
                reciprocal_ranks.append(1 / rank)
                break
    return sum(reciprocal_ranks) / len(reciprocal_ranks)

# Example usage
recommended_items = {1: [101, 102, 103], 2: [104, 105, 106]}  # User ID: [Recommended Items]
relevant_items = {1: [102, 107], 2: [104, 108]}  # User ID: [Relevant Items]

mrr = mean_reciprocal_rank(recommended_items, relevant_items)
print(f'Mean Reciprocal Rank: {mrr}')

In [10]:
# Mean Average Precision (MAP) Both CF and CBF

from sklearn.metrics import average_precision_score

# Assuming `y_true` and `y_pred` are binary relevance vectors
map_score = average_precision_score(y_true, y_pred)
print(f'Mean Average Precision: {map_score}')

NameError: name 'y_true' is not defined

In [ ]:
# Normalized Discounted Cumulative Gain (nDCG) Both CF and CBF.

from sklearn.metrics import ndcg_score

# Assuming `y_true` is the relevance scores and `y_pred` is the predicted scores
ndcg = ndcg_score([y_true], [y_pred])
print(f'nDCG: {ndcg}')

In [ ]:
#  Cross-Validation Both CF and CBF.

from surprise.model_selection import cross_validate

# Assuming `data` is your dataset and `svd` is your model
results = cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

print(f'Average RMSE: {results["test_rmse"].mean()}')
print(f'Average MAE: {results["test_mae"].mean()}')

In [ ]:
# Assuming you have a function `hybrid_recommend` that returns recommendations
# and a test set of user-movie interactions

def evaluate_hybrid_recommendations(user_ids, movie_titles, relevant_movies, top_n=10):
    precisions = []
    recalls = []
    for user_id, movie_title, relevant in zip(user_ids, movie_titles, relevant_movies):
        recommendations = hybrid_recommend(user_id, movie_title, top_n=top_n)
        y_true = [1 if movie in relevant else 0 for movie in all_movies]
        y_pred = [1 if movie in recommendations else 0 for movie in all_movies]
        precisions.append(precision_score(y_true, y_pred))
        recalls.append(recall_score(y_true, y_pred))

    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)
    f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)

    print(f'Average Precision: {avg_precision}')
    print(f'Average Recall: {avg_recall}')
    print(f'F1-Score: {f1}')

# Example usage
user_ids = [1, 2, 3]  # Example user IDs
movie_titles = ['Spider-Man', 'Inception', 'The Dark Knight']  # Example movie titles
relevant_movies = [['Spider-Man 2', 'Spider-Man 3'], ['Inception', 'Interstellar'], ['The Dark Knight Rises']]  # Example relevant movies

evaluate_hybrid_recommendations(user_ids, movie_titles, relevant_movies)